# Preparação dos Dados

Nesta etapa, faremos o redimensionamento de todas as imagens para garantir uniformidade no tamanho e realizaremos um pré-processamento das imagens para melhorar sua qualidade, ajustando o brilho e o contraste. Após essas modificações, salvaremos os dados modificados em um novo arquivo `.h5`.

## Importação das Bibliotecas e Conexão com o Drive

Primeiro, precisamos importar as bibliotecas necessárias e conectar ao Google Drive.

- **h5py**: Para salvar e carregar arquivos no formato HDF5, ideal para grandes volumes de dados.
- **cv2 (OpenCV)**: Utilizada para leitura, escrita e manipulação de imagens.
- **numpy (np)**: Biblioteca para manipulação eficiente de arrays e operações matemáticas.
- **matplotlib**: Permite plotar gráficos.
- **PIL (Python Imaging Library) e ImageEnhance:** PIL é uma biblioteca de manipulação de imagens. A classe `Image` permite carregar, modificar e salvar imagens, enquanto `ImageEnhance` fornece ferramentas para ajustar o brilho, contraste, nitidez, entre outros aspectos das imagens.
- **google.colab.drive**: Permite montar o Google Drive no Colab para acessar arquivos armazenados na nuvem.


In [56]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageEnhance
from google.colab import drive

# Conectando ao Google Drive
drive.mount('/content/drive')

# Importando o arquivo .h5 com os dados originais
h5_path = '/content/drive/MyDrive/data/combined_data.h5'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Função para Ajuste de Brilho e Contraste

Esta função realiza o ajuste de brilho e contraste nas imagens para melhorar a qualidade visual e facilitar a segmentação e classificação posterior.


In [57]:
def ajustar_brilho_contraste_pil(imagem_np, alpha=1.5, beta=1.1):
    # Convertendo o array NumPy para imagem PIL
    imagem_pil = Image.fromarray(imagem_np)

    # Ajustando o brilho
    enhancer_brightness = ImageEnhance.Brightness(imagem_pil)
    imagem_com_brilho = enhancer_brightness.enhance(beta)

    # Ajustando o contraste
    enhancer_contrast = ImageEnhance.Contrast(imagem_com_brilho)
    imagem_ajustada = enhancer_contrast.enhance(alpha)

    # Convertendo de volta para array NumPy
    imagem_final = np.array(imagem_ajustada)

    return imagem_final

## Carregamento das Imagens e Pré-processamento

Vamos carregar as imagens do arquivo `.h5`, redimensioná-las para um tamanho padrão e aplicar a função de ajuste de brilho e contraste.


In [58]:
# Abrindo o arquivo .h5
with h5py.File(h5_path, 'r') as h5f:
    X_data = h5f['X'][:]
    Y_data = h5f['Y'][:]

# Definindo o novo tamanho das imagens
novo_tamanho = (128, 128)

# Listas para armazenar as imagens processadas
X_modificado = []

# Exemplo de uso da função PIL
for i in range(len(X_data)):
    # Redimensionando as imagens X
    x_resized = cv2.resize(X_data[i], novo_tamanho, interpolation=cv2.INTER_LINEAR)

    # Aplicando ajuste de brilho e contraste usando PIL
    x_adjusted = ajustar_brilho_contraste_pil(x_resized)

    # Salvando a imagem processada
    X_modificado.append(x_adjusted)

# Convertendo a lista para um array numpy
X_modificado = np.array(X_modificado)


## Visualização das Imagens Modificadas

Agora, vamos visualizar algumas imagens para garantir que o pré-processamento foi feito corretamente. Serão mostrados o `X`, `X_modificado` e o `Y` correspondente.


In [59]:
# Exibindo as primeiras 20 imagens
for i in range(20):
    plt.figure(figsize=(12, 6))

    # Imagem original X
    plt.subplot(1, 3, 1)
    plt.imshow(X_data[i])
    plt.title(f'Imagem X {i+1}')

    # Imagem modificada X_modificado
    plt.subplot(1, 3, 2)
    plt.imshow(X_modificado[i])
    plt.title(f'Imagem Modificada {i+1}')

    # Imagem correspondente Y
    plt.subplot(1, 3, 3)
    plt.imshow(Y_data[i])
    plt.title(f'Imagem Y {i+1}')

    plt.show()


Output hidden; open in https://colab.research.google.com to view.

## Salvando os Dados Modificados

Após o pré-processamento, os dados serão salvos em um novo arquivo `.h5` em uma pasta separada dentro do Google Drive.


In [36]:
# Definindo o caminho para salvar o novo arquivo .h5
save_path = '/content/drive/MyDrive/data/modified/combined_data_modified.h5'

# Salvando os dados modificados em um novo arquivo .h5
with h5py.File(save_path, 'w') as h5f:
    h5f.create_dataset('X', data=X_modificado)
    h5f.create_dataset('Y', data=Y_data)
